In [1]:
!grep Commit /proc/meminfo

CommitLimit:     4073828 kB
Committed_AS:    2390688 kB


In [2]:
! pip3 install --upgrade kfp google-cloud-aiplatform google-cloud-storage google-cloud-pipeline-components==1.0.26

  Using cached kfp-2.13.0.tar.gz (269 kB)
  Preparing metadata (setup.py) ... done
  Using cached google_cloud_storage-3.1.0-py2.py3-none-any.whl.metadata (12 kB)


In [3]:
import random
import string
import os
import sys

import google.cloud.aiplatform as aip
from google.cloud import aiplatform


from google_cloud_pipeline_components.experimental.custom_job import utils

from kfp.v2 import compiler, dsl
# from kfp.v2.dsl import Dataset, Output, component, Artifact, ClassificationMetrics, Input, Metrics
from kfp.v2.dsl import component, OutputPath, pipeline, Input, Dataset, Output, Metrics

# Generate a uuid of a specifed length(default=8)

def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))

UUID = generate_uuid()





In [ ]:
# SERVICE_ACCOUNT = "simple-vertex-ai-account@coastal-cell-299117.iam.gserviceaccount.com"
SERVICE_ACCOUNT = "YOUR_SA"
PROJECT_ID = "YOUR_PROJECT_ID"
REGION = "us-central1"

BUCKET_NAME = "simple-vertex-ai-pipeline"
BUCKET_URI = f"gs://{BUCKET_NAME}"

# # upload model component 
# UPLOADED_MODEL_NAME = "model-uploaded-demo"

# pipeline
PIPELINE_ROOT = "gs://{}/pipeline_root/".format(BUCKET_NAME)
# PIPELINE_NAME = "custom-training-demo-pipeline"
# PIPELINE_JOB_NAME = "pipeline-job-name"
# COMPILED_PIPELINE_NAME = "custom_model_training_spec.json"

# 


In [5]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# Pipeline

In [6]:
from google_cloud_pipeline_components.v1.custom_job import create_custom_training_job_from_component
from google_cloud_pipeline_components.experimental.custom_job import utils

In [7]:
@component(
    packages_to_install=["pandas", "db-dtypes", "google-cloud-bigquery"],
    base_image="python:3.9",
    output_component_file="pipeline/preprocessing.yaml",
)
def preprocessing(
    query: str,
    output_data_path: str,
    project_id: str,
    bucket_name: str
):
    print("11111")

    print("Importing files")
    import pandas as pd
    import numpy as np
    from datetime import date, timedelta
    import time
    from google.cloud import bigquery
    from google.oauth2 import service_account
    import subprocess
    from google.cloud import storage
    from datetime import datetime, timedelta
    print("Syncing Files")
    print(query)
    print(output_data_path)
    print("current repo", subprocess.run("ls"))
    
    bqclient = bigquery.Client(project=project_id)
    
    query = "SELECT * FROM `coastal-cell-299117.promptTutorTutorial.bikedemand` LIMIT 10"
    # print("Querying from bigquery ...")
    dataframe= bqclient.query(query).to_dataframe()
    dataframe.to_csv(f'/gcs/{bucket_name}/{output_data_path}', index=False)



custom_component_1 = utils.create_custom_training_job_op_from_component(
    # custom_train_model, replica_count=1
    preprocessing,
    display_name = 'custom_component_1',
    machine_type = 'n2-highmem-8',
    # accelerator_type='Nvidia Tesla P100 GPU',
    # accelerator_count='1'
)




In [8]:
@component(
    packages_to_install=["pandas", "db-dtypes", "google-cloud-bigquery"],
    base_image="python:3.9",
    output_component_file="pipeline/preprocessing.yaml",
)
def preprocessing_2(
    input_data_path: str,
    output_data_path: str,
    project_id: str,
    bucket_name: str
):
    print("11111")

    print("Importing files")
    import pandas as pd
    import numpy as np
    from datetime import date, timedelta
    import time
    from google.cloud import bigquery
    from google.oauth2 import service_account
    import subprocess
    from google.cloud import storage
    from datetime import datetime, timedelta
    
    dataframe = pd.read_csv(f'/gcs/{bucket_name}/{input_data_path}')
    dataframe.to_csv(f'/gcs/{bucket_name}/{output_data_path}')
    print(dataframe)



custom_component_2 = utils.create_custom_training_job_op_from_component(
    # custom_train_model, replica_count=1
    preprocessing_2,
    display_name = 'custom_component_2',
    machine_type = 'n2-highmem-8',
    # accelerator_type='Nvidia Tesla P100 GPU',
    # accelerator_count='1'
)




In [9]:
import pandas as pd
from datetime import timedelta
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline.
    name="mews-pipeline-test-1",
)
def pipeline(
    query: str,
    input_data_path_1: str,
    output_data_path_1: str,
    output_data_path_2: str,
    bucket_name: str,
    # output_data_path: str = "VeryDenseDataset.csv",
):  
    custom_component_1_task = custom_component_1(    
        project=PROJECT_ID,
        location=REGION,
        query=query,
        output_data_path=output_data_path_1,
        project_id=PROJECT_ID,
        bucket_name=BUCKET_NAME
        # base_output_directory=PIPELINE_ROOT,
    )
    custom_component_2_task = custom_component_2(    
        project=PROJECT_ID,
        location=REGION,
        input_data_path=input_data_path_1,
        output_data_path=output_data_path_2,
        project_id=PROJECT_ID,
        bucket_name=BUCKET_NAME
        # base_output_directory=PIPELINE_ROOT,
    )
    custom_component_2_task.after(custom_component_1_task)

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="pipeline/vertex-ai-test-pipeline.json"
)

/opt/conda/lib/python3.10/site-packages/kfp/v2/compiler/compiler.py:1290: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


In [11]:
run1 = aip.PipelineJob(
    display_name="production-simple-pipeline-test",
    template_path="pipeline/vertex-ai-test-pipeline.json",
    # job_id="test-pipeline34",
    parameter_values={
        "query": "Input string",
        "output_data_path_1": "bikedataset.csv",
        "input_data_path_1": "bikedataset.csv",
        "output_data_path_2": "bikedataset_2.csv",
        "bucket_name": BUCKET_NAME
    },
    enable_caching=False,
)

run1.run(service_account=SERVICE_ACCOUNT)


Creating PipelineJob
PipelineJob created. Resource name: projects/925388845471/locations/us-central1/pipelineJobs/mews-pipeline-test-1-20250503065026
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/925388845471/locations/us-central1/pipelineJobs/mews-pipeline-test-1-20250503065026')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/mews-pipeline-test-1-20250503065026?project=925388845471
PipelineJob projects/925388845471/locations/us-central1/pipelineJobs/mews-pipeline-test-1-20250503065026 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/925388845471/locations/us-central1/pipelineJobs/mews-pipeline-test-1-20250503065026 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/925388845471/locations/us-central1/pipelineJobs/mews-pipeline-test-1-20250503065026 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/925388845471/locatio